In [ ]:
import sib_api_v3_sdk
from sib_api_v3_sdk.rest import ApiException
import dotenv
import os
import pandas as pd
dotenv.load_dotenv(dotenv.find_dotenv())

In [ ]:
# Get the list of emails from a one-column CSV
EMAIL_FILE_PATH = os.getenv("RGPD_EMAIL_CSV_FILE_PATH")

# The source where the emails come from (a pipeline source ID)
EMAIL_SOURCE=os.getenv('RGPD_EMAIL_SOURCE')

df_emails = pd.read_csv(EMAIL_FILE_PATH, index_col=None, dtype=str)
df_emails["SOURCE"] = EMAIL_SOURCE
print(df_emails.info())

emails_as_csv = df_emails.to_csv(index=None, sep=";")
print(emails_as_csv)

In [ ]:
# IMPORTING CONTACTS INTO BREVO
# https://app.brevo.com/contact/list/id/77

configuration = sib_api_v3_sdk.Configuration()
configuration.api_key['api-key'] = os.environ["BREVO_API_KEY"]

# Check the API key is available (should be in analyse/.env at BREVO_API_KEY)
assert configuration.api_key['api-key'] != ""

# Documentation for import contacts: https://developers.brevo.com/reference/importcontacts-1
api_instance = sib_api_v3_sdk.ContactsApi(sib_api_v3_sdk.ApiClient(configuration))
request_contact_import = sib_api_v3_sdk.RequestContactImport()
request_contact_import.file_body = emails_as_csv
request_contact_import.list_ids = [77]
request_contact_import.email_blacklist = False
request_contact_import.sms_blacklist = False
request_contact_import.update_existing_contacts = True
request_contact_import.empty_contacts_attributes = False

try:
    api_response = api_instance.import_contacts(request_contact_import)
    print(api_response)
except ApiException as e:
    print("Exception when calling ContactsApi->import_contacts: %s\n" % e)